In [1]:
from bayes_opt import BayesianOptimization
from matplotlib.pyplot i

In [5]:
oneD_function = lambda x : -x ** 2 -  x** 2 + 1

In [7]:
bo = BayesianOptimization(oneD_function,
                          {'x': (-4, 4)})